<a href="https://colab.research.google.com/github/SharmaAshwini/UNT/blob/main/Methods%20in%20Empirical%20Analysis/Exercise_2_Text_Summarization_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import all necessary libraries

In [ ]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

import nltk
nltk.download('stopwords')

import nltk
nltk.download('punkt')
nltk.download('wordnet')

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


#Generate clean sentences (Tutorial)

In [ ]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
     #print(sentence)
     sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()
    #print(sentences)
    return sentences

#Read and perform text cleanup (not from Tutorial)

Read Text

In [ ]:
def read_article_all_cleanup(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
     sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()

    ##Change case to lower
    sentences_lower = [[string.lower() for string in sublist] for sublist in sentences]

    ##Punctuation
    punctuation_signs = list("?:!.,;")
    for punct_sign in punctuation_signs:
      sentences_lower_punct = [[string.replace(punct_sign, '') for string in sublist] for sublist in sentences_lower]

    ##Remove possessive pronoun terminations
    sentences_lower_punct_pro = [[string.replace("'s", "") for string in sublist] for sublist in sentences_lower_punct]

    ##Lemmatization
    # Saving the lemmatizer into an object
    wordnet_lemmatizer = WordNetLemmatizer()

    lemmatized_text_list = []
    for i in range(len(sentences_lower_punct)):
      lemmatized_list = []
      for j in range(len(sentences_lower_punct[i])):
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(sentences_lower_punct[i][j], pos="v"))

      # Append to the list containing the texts
      lemmatized_text_list.append(lemmatized_list)

    return sentences, sentences_lower, sentences_lower_punct, sentences_lower_punct_pro, lemmatized_text_list

Read from File

In [ ]:
def read_and_clean_file(file_name):
  file = open(file_name, "r")
  filedata = file.readlines()
  artc = filedata[0].split(". ")
  length_file = len(artc)
  statement = []
  for i in range(length_file):
    article = artc[i]
    statement.append(article.split())

  ##Remove \r, \n and nulls, generate list of lists
  for i in range(len(statement)):
    for j in range(len(statement[i])):
      if statement[i][j] in ["\r", "\n","","    "]:
        statement[i][j] = " "
      if statement[i][j] in ['"']:
        statement[i][j] = ""

    while(" " in statement[i]):
      statement[i].remove(" ")

  while [] in statement:
    statement.remove([])

  ##Change case to lower
  statement = [[string.lower() for string in sublist] for sublist in statement]

  ##Punctuation
  punctuation_signs = list("?:!.,;")
  for punct_sign in punctuation_signs:
    statement = [[string.replace(punct_sign, '') for string in sublist] for sublist in statement]

  ##Remove possessive pronoun terminations
  statement = [[string.replace("'s", "") for string in sublist] for sublist in statement]

  ##Lemmatization
  # Saving the lemmatizer into an object
  wordnet_lemmatizer = WordNetLemmatizer()

  lemmatized_text_list = []
  for i in range(len(statement)):
    lemmatized_list = []
    for j in range(len(statement[i])):
      lemmatized_list.append(wordnet_lemmatizer.lemmatize(statement[i][j], pos="v"))

    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_list)

  return lemmatized_text_list

#Read and perform text cleanup (not from Tutorial; can handle text files with new lines)

In [ ]:
def read_and_clean_file_nl(file_name):
  file = open(file_name, "r")
  filedata = file.readlines()
  article = filedata[0].split(". ")
  length_file = len(filedata)
  statement = []
  for i in range(length_file):
    article = filedata[i]
    statement.append(article.split())

  ##Remove \r, \n and nulls, generate list of lists
  for i in range(len(statement)):
    for j in range(len(statement[i])):
      if statement[i][j] in ["\r", "\n","","    "]:
        statement[i][j] = " "
      if statement[i][j] in ['"']:
        statement[i][j] = ""

    while(" " in statement[i]):
      statement[i].remove(" ")

  while [] in statement:
    statement.remove([])

  ##Change case to lower
  statement = [[string.lower() for string in sublist] for sublist in statement]

  ##Punctuation
  punctuation_signs = list("?:!.,;")
  for punct_sign in punctuation_signs:
    statement = [[string.replace(punct_sign, '') for string in sublist] for sublist in statement]

  ##Remove possessive pronoun terminations
  statement = [[string.replace("'s", "") for string in sublist] for sublist in statement]

  ##Lemmatization
  # Saving the lemmatizer into an object
  wordnet_lemmatizer = WordNetLemmatizer()

  lemmatized_text_list = []
  for i in range(len(statement)):
    lemmatized_list = []
    for j in range(len(statement[i])):
      lemmatized_list.append(wordnet_lemmatizer.lemmatize(statement[i][j], pos="v"))

    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_list)

  return lemmatized_text_list

#Similarity matrix

In [ ]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

In [ ]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

#Generate Summary Method (Tutorial)

In [ ]:
def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text and tokenize
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

#Generate Summary Method (with cleanUp)

In [ ]:
def generate_clean_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text and tokenize
    sentences =  read_and_clean_file(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

#Generate Summary Method (with all cleanUp outputs)

In [ ]:
def generate_all_clean_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    summarize_text_lower = []
    summarize_text_lower_punct = []
    summarize_text_lower_punct_pro = []
    summarize_text_lemmatized = []

    # Step 1 - Read text and tokenize
    sentences, sentences_lower, sentences_lower_punct, sentences_lower_punct_pro, lemmatized_text_list =  read_article_all_cleanup(file_name)


    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    sentence_similarity_martix_lower = build_similarity_matrix(sentences_lower, stop_words)
    sentence_similarity_martix_lower_punct = build_similarity_matrix(sentences_lower_punct, stop_words)
    sentence_similarity_martix_lower_punct_pro = build_similarity_matrix(sentences_lower_punct_pro, stop_words)
    sentence_similarity_martix_lemmatized = build_similarity_matrix(lemmatized_text_list, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    sentence_similarity_graph_lower = nx.from_numpy_array(sentence_similarity_martix_lower)
    scores_lower = nx.pagerank(sentence_similarity_graph_lower)

    sentence_similarity_graph_lower_punct = nx.from_numpy_array(sentence_similarity_martix_lower_punct)
    scores_lower_punct = nx.pagerank(sentence_similarity_graph_lower_punct)

    sentence_similarity_graph_lower_punct_pro = nx.from_numpy_array(sentence_similarity_martix_lower_punct_pro)
    scores_lower_punct_pro = nx.pagerank(sentence_similarity_graph_lower_punct_pro)

    sentence_similarity_graph_lemmatized = nx.from_numpy_array(sentence_similarity_martix_lemmatized)
    scores_lemmatized = nx.pagerank(sentence_similarity_graph_lemmatized)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are ", ranked_sentence)

    ranked_sentence_lower = sorted(((scores_lower[i],s) for i,s in enumerate(sentences_lower)), reverse=True)
    print("Indexes of top ranked_sentence (with lower case) order are ", ranked_sentence_lower)

    ranked_sentence_lower_punct = sorted(((scores_lower_punct[i],s) for i,s in enumerate(sentences_lower_punct)), reverse=True)
    print("Indexes of top ranked_sentence (with lower case & punctuation) order are ", ranked_sentence_lower_punct)

    ranked_sentence_lower_punct_pro = sorted(((scores_lower_punct_pro[i],s) for i,s in enumerate(sentences_lower_punct_pro)), reverse=True)
    print("Indexes of top ranked_sentence (with lower case, punctuation & pronoun) order are ", ranked_sentence_lower_punct_pro)

    ranked_sentence_lemmatized = sorted(((scores_lemmatized[i],s) for i,s in enumerate(lemmatized_text_list)), reverse=True)
    print("Indexes of top ranked_sentence (with lower case, punctuation, pronoun & lemmatization) order are ", ranked_sentence_lemmatized)

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

      summarize_text_lower.append(" ".join(ranked_sentence_lower[i][1]))
      summarize_text_lower_punct.append(" ".join(ranked_sentence_lower_punct[i][1]))
      summarize_text_lower_punct_pro.append(" ".join(ranked_sentence_lower_punct_pro[i][1]))
      summarize_text_lemmatized.append(" ".join(ranked_sentence_lemmatized[i][1]))


    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text), "\n")
    print("Summarize Text (with lower case): \n", ". ".join(summarize_text_lower), "\n")
    print("Summarize Text (with lower case & punctuation): \n", ". ".join(summarize_text_lower_punct), "\n")
    print("Summarize Text (with lower case, punctuation & pronoun): \n", ". ".join(summarize_text_lower_punct_pro), "\n")
    print("Summarize Text (with lower case, punctuation, pronoun & lemmatization): \n", ". ".join(summarize_text_lemmatized), "\n")

#Generate Summary Method (with blogs cleanUp)

In [ ]:
def generate_clean_summary_blog(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text and tokenize
    sentences =  read_and_clean_file_nl(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

##Test data

In [ ]:
!echo "In an attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills. Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services. As part of the program, the Redmond giant which wants to expand its reach and is planning to build a strong developer ecosystem in India with the program will set up the core AI infrastructure and IoT Hub for the selected campuses. The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning.According to Manish Prakash, Country General Manager-PS, Health and Education, Microsoft India, said, "With AI being the defining technology of our time, it is transforming lives and industry and the jobs of tomorrow will require a different skillset. This will require more collaborations and training and working with AI. That’s why it has become more critical than ever for educational institutions to integrate new cloud and AI technologies. The program is an attempt to ramp up the institutional set-up and build capabilities among the educators to educate the workforce of tomorrow." The program aims to build up the cognitive skills and in-depth understanding of developing intelligent cloud connected solutions for applications across industry. Earlier in April this year, the company announced Microsoft Professional Program In AI as a learning track open to the public. The program was developed to provide job ready skills to programmers who wanted to hone their skills in AI and data science with a series of online courses which featured hands-on labs and expert instructors as well. This program also included developer-focused AI school that provided a bunch of assets to help build AI skills." > msft.txt

In [ ]:
!echo "For years, Facebook gave some of the world's largest technology companies more intrusive access to users' personal data than it has disclosed, effectively exempting those business partners from its usual privacy rules, according to internal records and interviews. The special arrangements are detailed in hundreds of pages of Facebook documents obtained by The New York Times. The records, generated in 2017 by the company's internal system for tracking partnerships, provide the most complete picture yet of the social network's data-sharing practices. They also underscore how personal data has become the most prized commodity of the digital age, traded on a vast scale by some of the most powerful companies in Silicon Valley and beyond. The exchange was intended to benefit everyone. Pushing for explosive growth, Facebook got more users, lifting its advertising revenue. Partner companies acquired features to make their products more attractive. Facebook users connected with friends across different devices and websites. But Facebook also assumed extraordinary power over the personal information of its 2 billion users - control it has wielded with little transparency or outside oversight.Facebook allowed Microsoft's Bing search engine to see the names of virtually all Facebook user's friends without consent, the records show, and gave Netflix and Spotify the ability to read Facebook users' private messages." > fb.txt

In [ ]:
!echo "WASHINGTON - The Trump administration has ordered the military to start withdrawing roughly 7,000 troops from Afghanistan in the coming months, two defense officials said Thursday, an abrupt shift in the 17-year-old war there and a decision that stunned Afghan officials, who said they had not been briefed on the plans.President Trump made the decision to pull the troops - about half the number the United States has in Afghanistan now - at the same time he decided to pull American forces out of Syria, one official said.The announcement came hours after Jim Mattis, the secretary of defense, said that he would resign from his position at the end of February after disagreeing with the president over his approach to policy in the Middle East.The whirlwind of troop withdrawals and the resignation of Mr. Mattis leave a murky picture for what is next in the United States’ longest war, and they come as Afghanistan has been troubled by spasms of violence afflicting the capital, Kabul, and other important areas. The United States has also been conducting talks with representatives of the Taliban, in what officials have described as discussions that could lead to formal talks to end the conflict.Senior Afghan officials and Western diplomats in Kabul woke up to the shock of the news on Friday morning, and many of them braced for chaos ahead. Several Afghan officials, often in the loop on security planning and decision-making, said they had received no indication in recent days that the Americans would pull troops out. The fear that Mr. Trump might take impulsive actions, however, often loomed in the background of discussions with the United States, they said.They saw the abrupt decision as a further sign that voices from the ground were lacking in the debate over the war and that with Mr. Mattis’s resignation, Afghanistan had lost one of the last influential voices in Washington who channeled the reality of the conflict into the White House’s deliberations.The president long campaigned on bringing troops home, but in 2017, at the request of Mr. Mattis, he begrudgingly pledged an additional 4,000 troops to the Afghan campaign to try to hasten an end to the conflict.Though Pentagon officials have said the influx of forces - coupled with a more aggressive air campaign - was helping the war effort, Afghan forces continued to take nearly unsustainable levels of casualties and lose ground to the Taliban.The renewed American effort in 2017 was the first step in ensuring Afghan forces could become more independent without a set timeline for a withdrawal. But with plans to quickly reduce the number of American troops in the country, it is unclear if the Afghans can hold their own against an increasingly aggressive Taliban.Currently, American airstrikes are at levels not seen since the height of the war, when tens of thousands of American troops were spread throughout the country. That air support, officials say, consists mostly of propping up Afghan troops while they try to hold territory from a resurgent Taliban." > trump.txt

In [ ]:
!echo "The biggest rumor heading into this year's WWDC is of course the Apple headset. Rumored to be running on a new "xrOS," the device could utilize mixed reality, a combination of virtual reality and augmented reality. There may be eye and hand tracking, high-resolution displays and... a potential $3,000 price tag. Bloomberg's Mark Gurman recently detailed how Apple plans to incorporate sports, gaming, workouts and iPad apps into the headset to show off what the new platform can do. Whether that's enough to excite consumers and persuade them to drop three grand or for developers to commit to building apps for it remains to be seen. The MacBook Air has long been one of Apple's most popular laptops. Frequently sold with a 13-inch screen, Apple has experimented with different sizes of Airs in the past, including offering an 11-inch model for years. Rumors these days, however, suggest that the company has a larger, 15-inch M2-powered Air raring to go. That once again comes from Bloomberg's Gurman, who expects the new laptop to be announced at this year's event. It's about time. While rumors point to an imminent announcement, it's unclear how much Apple might charge for the new Air or how it might fit into the company's existing MacBook lineup. The 2020 M1-powered 13.3-inch MacBook Air is still sold for $999, while the updated M2-powered 2022 13.6-inch MacBook Air starts at $1,199. A 16-inch MacBook Pro, meanwhile, starts at $2,499. Might the 15-inch Air fit somewhere in the middle? Oh, the Mac Pro. Apple last updated the Mac Pro at WWDC in 2019. Despite some teases that confirmed it's working on a new one powered by its Apple Silicon chips, the company has largely been quiet about the super powerful computer. Might the "another day" be June 5? It's possible and Mac Pro fans may want to tune in, but with tempered expectations. In an April appearance on The MacRumors Show, Gurman, the Apple savant, suggests that it still may arrive this year but not at WWDC. In addition to all the hardware rumors, we can expect Apple to detail the latest updates coming this year to its iOS, iPadOS, WatchOS and TVOS platforms. Among the bigger iOS changes, Apple might finally add support for installing apps not downloaded from the App Store. The iPhone maker has long resisted opening up its mobile software to allow for sideloading, but new European regulations may have forced its hand. Other software changes Apple might unveil include a new mental health app as well as widgets returning to the Apple Watch." > apple_mwc.txt

##Test Runs

### generate summary function is from tutorial and generate_clean_summary has text cleanup

Test from tutorial and with CleanUp

In [ ]:
generate_summary( "msft.txt", 2)

Indexes of top ranked_sentence order are  [(0.1357055978868266, ['Envisioned', 'as', 'a', 'three-year', 'collaborative', 'program,', 'Intelligent', 'Cloud', 'Hub', 'will', 'support', 'around', '100', 'institutions', 'with', 'AI', 'infrastructure,', 'course', 'content', 'and', 'curriculum,', 'developer', 'support,', 'development', 'tools', 'and', 'give', 'students', 'access', 'to', 'cloud', 'and', 'AI', 'services']), (0.12193277895857677, ['The', 'company', 'will', 'provide', 'AI', 'development', 'tools', 'and', 'Azure', 'AI', 'services', 'such', 'as', 'Microsoft', 'Cognitive', 'Services,', 'Bot', 'Services', 'and', 'Azure', 'Machine', 'Learning.According', 'to', 'Manish', 'Prakash,', 'Country', 'General', 'Manager-PS,', 'Health', 'and', 'Education,', 'Microsoft', 'India,', 'said,', 'With', 'AI', 'being', 'the', 'defining', 'technology', 'of', 'our', 'time,', 'it', 'is', 'transforming', 'lives', 'and', 'industry', 'and', 'the', 'jobs', 'of', 'tomorrow', 'will', 'require', 'a', 'differen

In [ ]:
generate_clean_summary("msft.txt", 2)

Indexes of top ranked_sentence order are  [(0.13129671766785161, ['this', 'program', 'also', 'include', 'developer-focused', 'ai', 'school', 'that', 'provide', 'a', 'bunch', 'of', 'assets', 'to', 'help', 'build', 'ai', 'skills']), (0.11877677193211429, ['envision', 'as', 'a', 'three-year', 'collaborative', 'program', 'intelligent', 'cloud', 'hub', 'will', 'support', 'around', '100', 'institutions', 'with', 'ai', 'infrastructure', 'course', 'content', 'and', 'curriculum', 'developer', 'support', 'development', 'tool', 'and', 'give', 'students', 'access', 'to', 'cloud', 'and', 'ai', 'service']), (0.09870938464353562, ['as', 'part', 'of', 'the', 'program', 'the', 'redmond', 'giant', 'which', 'want', 'to', 'expand', 'its', 'reach', 'and', 'be', 'plan', 'to', 'build', 'a', 'strong', 'developer', 'ecosystem', 'in', 'india', 'with', 'the', 'program', 'will', 'set', 'up', 'the', 'core', 'ai', 'infrastructure', 'and', 'iot', 'hub', 'for', 'the', 'select', 'campuses']), (0.09741236591123935, ['t

Test for different CleanUps

In [ ]:
generate_all_clean_summary("msft.txt", 2)

Indexes of top ranked_sentence order are  [(0.1357055978868266, ['Envisioned', 'as', 'a', 'three-year', 'collaborative', 'program,', 'Intelligent', 'Cloud', 'Hub', 'will', 'support', 'around', '100', 'institutions', 'with', 'AI', 'infrastructure,', 'course', 'content', 'and', 'curriculum,', 'developer', 'support,', 'development', 'tools', 'and', 'give', 'students', 'access', 'to', 'cloud', 'and', 'AI', 'services']), (0.12193277895857677, ['The', 'company', 'will', 'provide', 'AI', 'development', 'tools', 'and', 'Azure', 'AI', 'services', 'such', 'as', 'Microsoft', 'Cognitive', 'Services,', 'Bot', 'Services', 'and', 'Azure', 'Machine', 'Learning.According', 'to', 'Manish', 'Prakash,', 'Country', 'General', 'Manager-PS,', 'Health', 'and', 'Education,', 'Microsoft', 'India,', 'said,', 'With', 'AI', 'being', 'the', 'defining', 'technology', 'of', 'our', 'time,', 'it', 'is', 'transforming', 'lives', 'and', 'industry', 'and', 'the', 'jobs', 'of', 'tomorrow', 'will', 'require', 'a', 'differen

Comparison for different texts

In [ ]:
generate_summary( "trump.txt", 2)

Indexes of top ranked_sentence order are  [(0.21481805670368562, ['WASHINGTON', '-', 'The', 'Trump', 'administration', 'has', 'ordered', 'the', 'military', 'to', 'start', 'withdrawing', 'roughly', '7,000', 'troops', 'from', 'Afghanistan', 'in', 'the', 'coming', 'months,', 'two', 'defense', 'officials', 'said', 'Thursday,', 'an', 'abrupt', 'shift', 'in', 'the', '17-year-old', 'war', 'there', 'and', 'a', 'decision', 'that', 'stunned', 'Afghan', 'officials,', 'who', 'said', 'they', 'had', 'not', 'been', 'briefed', 'on', 'the', 'plans.President', 'Trump', 'made', 'the', 'decision', 'to', 'pull', 'the', 'troops', '-', 'about', 'half', 'the', 'number', 'the', 'United', 'States', 'has', 'in', 'Afghanistan', 'now', '-', 'at', 'the', 'same', 'time', 'he', 'decided', 'to', 'pull', 'American', 'forces', 'out', 'of', 'Syria,', 'one', 'official', 'said.The', 'announcement', 'came', 'hours', 'after', 'Jim', 'Mattis,', 'the', 'secretary', 'of', 'defense,', 'said', 'that', 'he', 'would', 'resign', 'fr

In [ ]:
generate_clean_summary( "trump.txt", 2)

Indexes of top ranked_sentence order are  [(0.17825073056841495, ['washington', '-', 'the', 'trump', 'administration', 'have', 'order', 'the', 'military', 'to', 'start', 'withdraw', 'roughly', '7000', 'troop', 'from', 'afghanistan', 'in', 'the', 'come', 'months', 'two', 'defense', 'officials', 'say', 'thursday', 'an', 'abrupt', 'shift', 'in', 'the', '17-year-old', 'war', 'there', 'and', 'a', 'decision', 'that', 'stun', 'afghan', 'officials', 'who', 'say', 'they', 'have', 'not', 'be', 'brief', 'on', 'the', 'planspresident', 'trump', 'make', 'the', 'decision', 'to', 'pull', 'the', 'troop', '-', 'about', 'half', 'the', 'number', 'the', 'unite', 'state', 'have', 'in', 'afghanistan', 'now', '-', 'at', 'the', 'same', 'time', 'he', 'decide', 'to', 'pull', 'american', 'force', 'out', 'of', 'syria', 'one', 'official', 'saidthe', 'announcement', 'come', 'hours', 'after', 'jim', 'mattis', 'the', 'secretary', 'of', 'defense', 'say', 'that', 'he', 'would', 'resign', 'from', 'his', 'position', 'at',

In [ ]:
generate_summary( "fb.txt", 2)

Indexes of top ranked_sentence order are  [(0.25755053984117615, ['For', 'years,', 'Facebook', 'gave', 'some', 'of', 'the', "world's", 'largest', 'technology', 'companies', 'more', 'intrusive', 'access', 'to', "users'", 'personal', 'data', 'than', 'it', 'has', 'disclosed,', 'effectively', 'exempting', 'those', 'business', 'partners', 'from', 'its', 'usual', 'privacy', 'rules,', 'according', 'to', 'internal', 'records', 'and', 'interviews']), (0.15321342634501522, ['Facebook', 'users', 'connected', 'with', 'friends', 'across', 'different', 'devices', 'and', 'websites']), (0.1484419990863834, ['Pushing', 'for', 'explosive', 'growth,', 'Facebook', 'got', 'more', 'users,', 'lifting', 'its', 'advertising', 'revenue']), (0.14021994428057658, ['The', 'special', 'arrangements', 'are', 'detailed', 'in', 'hundreds', 'of', 'pages', 'of', 'Facebook', 'documents', 'obtained', 'by', 'The', 'New', 'York', 'Times']), (0.1351454272881601, ['They', 'also', 'underscore', 'how', 'personal', 'data', 'has',

In [ ]:
generate_clean_summary( "fb.txt", 2)

Indexes of top ranked_sentence order are  [(0.1863994516865572, ['for', 'years', 'facebook', 'give', 'some', 'of', 'the', 'world', 'largest', 'technology', 'company', 'more', 'intrusive', 'access', 'to', "users'", 'personal', 'data', 'than', 'it', 'have', 'disclose', 'effectively', 'exempt', 'those', 'business', 'partner', 'from', 'its', 'usual', 'privacy', 'rule', 'accord', 'to', 'internal', 'record', 'and', 'interview']), (0.18300916139630743, ['but', 'facebook', 'also', 'assume', 'extraordinary', 'power', 'over', 'the', 'personal', 'information', 'of', 'its', '2', 'billion', 'users', '-', 'control', 'it', 'have', 'wield', 'with', 'little', 'transparency', 'or', 'outside', 'oversightfacebook', 'allow', 'microsoft', 'bing', 'search', 'engine', 'to', 'see', 'the', 'name', 'of', 'virtually', 'all', 'facebook', 'user', 'friends', 'without', 'consent', 'the', 'record', 'show', 'and', 'give', 'netflix', 'and', 'spotify', 'the', 'ability', 'to', 'read', 'facebook', "users'", 'private', 'mes

In [ ]:
generate_summary( "apple_mwc.txt", 2)

Indexes of top ranked_sentence order are  [(0.09501693609795944, ['While', 'rumors', 'point', 'to', 'an', 'imminent', 'announcement,', "it's", 'unclear', 'how', 'much', 'Apple', 'might', 'charge', 'for', 'the', 'new', 'Air', 'or', 'how', 'it', 'might', 'fit', 'into', 'the', "company's", 'existing', 'MacBook', 'lineup']), (0.07572644199422097, ['Apple', 'last', 'updated', 'the', 'Mac', 'Pro', 'at', 'WWDC', 'in', '2019']), (0.06673847113933327, ['The', '2020', 'M1-powered', '13.3-inch', 'MacBook', 'Air', 'is', 'still', 'sold', 'for', '999,', 'while', 'the', 'updated', 'M2-powered', '2022', '13.6-inch', 'MacBook', 'Air', 'starts', 'at', '1,199']), (0.06587142183034832, ['In', 'an', 'April', 'appearance', 'on', 'The', 'MacRumors', 'Show,', 'Gurman,', 'the', 'Apple', 'savant,', 'suggests', 'that', 'it', 'still', 'may', 'arrive', 'this', 'year', 'but', 'not', 'at', 'WWDC']), (0.06583552689160813, ['Might', 'the', '15-inch', 'Air', 'fit', 'somewhere', 'in', 'the', 'middle?', 'Oh,', 'the', 'Ma

In [ ]:
generate_clean_summary( "apple_mwc.txt", 2)

Indexes of top ranked_sentence order are  [(0.0802774753894062, ['while', 'rumor', 'point', 'to', 'an', 'imminent', 'announcement', 'it', 'unclear', 'how', 'much', 'apple', 'might', 'charge', 'for', 'the', 'new', 'air', 'or', 'how', 'it', 'might', 'fit', 'into', 'the', 'company', 'exist', 'macbook', 'lineup']), (0.07009732384838405, ['other', 'software', 'change', 'apple', 'might', 'unveil', 'include', 'a', 'new', 'mental', 'health', 'app', 'as', 'well', 'as', 'widgets', 'return', 'to', 'the', 'apple', 'watch']), (0.06885837412975318, ['the', 'macbook', 'air', 'have', 'long', 'be', 'one', 'of', 'apple', 'most', 'popular', 'laptops']), (0.06435356252397864, ['apple', 'last', 'update', 'the', 'mac', 'pro', 'at', 'wwdc', 'in', '2019']), (0.06128384725192808, ['the', 'biggest', 'rumor', 'head', 'into', 'this', 'year', 'wwdc', 'be', 'of', 'course', 'the', 'apple', 'headset']), (0.06047299980677545, ['in', 'an', 'april', 'appearance', 'on', 'the', 'macrumors', 'show', 'gurman', 'the', 'apple

## Can summarize paragraphs as well (from ICE-1)

In [ ]:
import requests

response = requests.get('https://raw.githubusercontent.com/miguelfzafra/Latest-News-Classifier/master/0.%20Latest%20News%20Classifier/00.%20Raw%20dataset/BBC/bbc-fulltext/bbc/business/001.txt')
data = response.text
print(data,  file=open('business_001.txt', 'w'))

response = requests.get('https://raw.githubusercontent.com/miguelfzafra/Latest-News-Classifier/master/0.%20Latest%20News%20Classifier/00.%20Raw%20dataset/BBC/bbc-fulltext/bbc/business/002.txt')
data = response.text
print(data,  file=open('business_002.txt', 'w'))

response = requests.get('https://raw.githubusercontent.com/miguelfzafra/Latest-News-Classifier/master/0.%20Latest%20News%20Classifier/00.%20Raw%20dataset/BBC/bbc-fulltext/bbc/business/003.txt')
data = response.text
print(data,  file=open('business_003.txt', 'w'))

response = requests.get('https://raw.githubusercontent.com/miguelfzafra/Latest-News-Classifier/master/0.%20Latest%20News%20Classifier/00.%20Raw%20dataset/BBC/bbc-fulltext/bbc/business/004.txt')
data = response.text
print(data,  file=open('business_004.txt', 'w'))

response = requests.get('https://raw.githubusercontent.com/miguelfzafra/Latest-News-Classifier/master/0.%20Latest%20News%20Classifier/00.%20Raw%20dataset/BBC/bbc-fulltext/bbc/business/005.txt')
data = response.text
print(data,  file=open('business_005.txt', 'w'))

In [ ]:
generate_clean_summary_blog("business_001.txt", 2)

Indexes of top ranked_sentence order are  [(0.21293950793362676, ['the', 'firm', 'which', 'be', 'now', 'one', 'of', 'the', 'biggest', 'investors', 'in', 'google', 'benefit', 'from', 'sales', 'of', 'high-speed', 'internet', 'connections', 'and', 'higher', 'advert', 'sales', 'timewarner', 'say', 'fourth', 'quarter', 'sales', 'rise', '2%', 'to', '$111bn', 'from', '$109bn', 'its', 'profit', 'be', 'buoy', 'by', 'one-off', 'gain', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'warner', 'bros', 'and', 'less', 'users', 'for', 'aol']), (0.2091791279867516, ['time', 'warner', 'say', 'on', 'friday', 'that', 'it', 'now', 'own', '8%', 'of', 'search-engine', 'google', 'but', 'its', 'own', 'internet', 'business', 'aol', 'have', 'have', 'mix', 'fortunes', 'it', 'lose', '464000', 'subscribers', 'in', 'the', 'fourth', 'quarter', 'profit', 'be', 'lower', 'than', 'in', 'the', 'precede', 'three', 'quarter', 'however', 'the', 'company', 'say', 'aol', 'underlie', 'profit', 'before', 'exceptional', 'items', 

In [ ]:
generate_clean_summary_blog("business_002.txt", 2)


Indexes of top ranked_sentence order are  [(0.32384532222678786, ['and', 'alan', 'greenspan', 'highlight', 'the', 'us', 'government', 'willingness', 'to', 'curb', 'spend', 'and', 'rise', 'household', 'save', 'as', 'factor', 'which', 'may', 'help', 'to', 'reduce', 'it', 'in', 'late', 'trade', 'in', 'new', 'york', 'the', 'dollar', 'reach', '$12871', 'against', 'the', 'euro', 'from', '$12974', 'on', 'thursday', 'market', 'concern', 'about', 'the', 'deficit', 'have', 'hit', 'the', 'greenback', 'in', 'recent', 'months', 'on', 'friday', 'federal', 'reserve', 'chairman', 'mr', 'greenspan', 'speech', 'in', 'london', 'ahead', 'of', 'the', 'meet', 'of', 'g7', 'finance', 'minister', 'send', 'the', 'dollar', 'higher', 'after', 'it', 'have', 'earlier', 'tumble', 'on', 'the', 'back', 'of', 'worse-than-expected', 'us', 'job', 'data', '"i', 'think', 'the', 'chairman', 'take', 'a', 'much', 'more', 'sanguine', 'view', 'on', 'the', 'current', 'account', 'deficit', 'than', 'he', 'take', 'for', 'some', 'ti

In [ ]:
generate_clean_summary_blog("business_003.txt", 2)

Indexes of top ranked_sentence order are  [(0.2874917615370608, ['yukos', 'unit', 'buyer', 'face', 'loan', 'claim']), (0.24285176610495757, ['state-owned', 'rosneft', 'buy', 'the', 'yugansk', 'unit', 'for', '$93bn', 'in', 'a', 'sale', 'force', 'by', 'russia', 'to', 'part', 'settle', 'a', '$275bn', 'tax', 'claim', 'against', 'yukos', "yukos'", 'owner', 'menatep', 'group', 'say', 'it', 'will', 'ask', 'rosneft', 'to', 'repay', 'a', 'loan', 'that', 'yugansk', 'have', 'secure', 'on', 'its', 'assets', 'rosneft', 'already', 'face', 'a', 'similar', '$540m', 'repayment', 'demand', 'from', 'foreign', 'bank', 'legal', 'experts', 'say', 'rosneft', 'purchase', 'of', 'yugansk', 'would', 'include', 'such', 'obligations', '"the', 'pledge', 'assets', 'be', 'with', 'rosneft', 'so', 'it', 'will', 'have', 'to', 'pay', 'real', 'money', 'to', 'the', 'creditors', 'to', 'avoid', 'seizure', 'of', 'yugansk', 'assets"', 'say', 'moscow-based', 'us', 'lawyer', 'jamie', 'firestone', 'who', 'be', 'not', 'connect', '

In [ ]:
generate_clean_summary_blog("business_004.txt", 2)

Indexes of top ranked_sentence order are  [(0.21801766775490167, ['high', 'fuel', 'price', 'hit', 'ba', 'profit']), (0.18227174422413359, ['to', 'help', 'offset', 'the', 'increase', 'price', 'of', 'aviation', 'fuel', 'ba', 'last', 'year', 'introduce', 'a', 'fuel', 'surcharge', 'for', 'passengers']), (0.16109831580936593, ['report', 'its', 'result', 'for', 'the', 'three', 'months', 'to', '31', 'december', '2004', 'the', 'airline', 'make', 'a', 'pre-tax', 'profit', 'of', '£75m', '($141m)', 'compare', 'with', '£125m', 'a', 'year', 'earlier', 'rod', 'eddington', 'ba', 'chief', 'executive', 'say', 'the', 'result', 'be', '"respectable"', 'in', 'a', 'third', 'quarter', 'when', 'fuel', 'cost', 'rise', 'by', '£106m', 'or', '473%', 'ba', 'profit', 'be', 'still', 'better', 'than', 'market', 'expectation', 'of', '£59m', 'and', 'it', 'expect', 'a', 'rise', 'in', 'full-year', 'revenues']), (0.1510443229773263, ['in', 'october', 'it', 'increase', 'this', 'from', '£6', 'to', '£10', 'one-way', 'for', '

In [ ]:
generate_clean_summary_blog("business_005.txt", 2)

Indexes of top ranked_sentence order are  [(0.2943405799012111, ['share', 'in', 'uk', 'drink', 'and', 'food', 'firm', 'ally', 'domecq', 'have', 'rise', 'on', 'speculation', 'that', 'it', 'could', 'be', 'the', 'target', 'of', 'a', 'takeover', 'by', 'france', 'pernod', 'ricard']), (0.24294286861867673, ['report', 'in', 'the', 'wall', 'street', 'journal', 'and', 'the', 'financial', 'time', 'suggest', 'that', 'the', 'french', 'spirit', 'firm', 'be', 'consider', 'a', 'bid', 'but', 'have', 'yet', 'to', 'contact', 'its', 'target', 'ally', 'domecq', 'share', 'in', 'london', 'rise', '4%', 'by', '1200', 'gmt', 'while', 'pernod', 'share', 'in', 'paris', 'slip', '12%', 'pernod', 'say', 'it', 'be', 'seek', 'acquisitions', 'but', 'refuse', 'to', 'comment', 'on', 'specifics']), (0.238022928422743, ['pernod', 'takeover', 'talk', 'lift', 'domecq']), (0.22469362305736895, ['pernod', 'last', 'major', 'purchase', 'be', 'a', 'third', 'of', 'us', 'giant', 'seagram', 'in', '2000', 'the', 'move', 'which', 'pr

#Summarization with TF-IDF

https://towardsdatascience.com/text-summarization-using-tf-idf-e64a0644ace3

In [ ]:
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def _create_frequency_table(text_string) -> dict:
    """
    we create a dictionary for the word frequency table.
    For this, we should only use the words that are not part of the stopWords array.

    Removing stop words and making frequency table
    Stemmer - an algorithm to bring words to its root word.
    :rtype: dict
    """
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable

In [ ]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [ ]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [ ]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [ ]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [ ]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [ ]:
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [ ]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average


In [ ]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [ ]:
def run_summarization(text):
    """
    :param text: Plain summary_text of long article
    :return: summarized summary_text
    """

    '''
    We already have a sentence tokenizer, so we just need
    to run the sent_tokenize() method to create the array of sentences.
    '''
    # 1 Sentence Tokenize
    sentences = sent_tokenize(text)
    total_documents = len(sentences)
    #print(sentences)

    # 2 Create the Frequency matrix of the words in each sentence.
    freq_matrix = _create_frequency_matrix(sentences)
    #print(freq_matrix)

    '''
    Term frequency (TF) is how often a word appears in a document, divided by how many words are there in a document.
    '''
    # 3 Calculate TermFrequency and generate a matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    '''
    Inverse document frequency (IDF) is how unique or rare a word is.
    '''
    # 5 Calculate IDF and generate a matrix
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(tf_idf_matrix)
    #print(sentence_scores)

    # 8 Find the threshold
    threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary
    summary = _generate_summary(sentences, sentence_scores, 1.3 * threshold)
    return summary

In [ ]:
text_str = '''
Those Who Are Resilient Stay In The Game Longer
“On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” — Friedrich Nietzsche
Challenges and setbacks are not meant to defeat you, but promote you. However, I realise after many years of defeats, it can crush your spirit and it is easier to give up than risk further setbacks and disappointments. Have you experienced this before? To be honest, I don’t have the answers. I can’t tell you what the right course of action is; only you will know. However, it’s important not to be discouraged by failure when pursuing a goal or a dream, since failure itself means different things to different people. To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Same failure, yet different responses. Who is right and who is wrong? Neither. Each person has a different mindset that decides their outcome. Those who are resilient stay in the game longer and draw on their inner means to succeed.

I’ve coached mummy and mom clients who gave up after many years toiling away at their respective goal or dream. It was at that point their biggest breakthrough came. Perhaps all those years of perseverance finally paid off. It was the 19th Century’s minister Henry Ward Beecher who once said: “One’s best success comes after their greatest disappointments.” No one knows what the future holds, so your only guide is whether you can endure repeated defeats and disappointments and still pursue your dream. Consider the advice from the American academic and psychologist Angela Duckworth who writes in Grit: The Power of Passion and Perseverance: “Many of us, it seems, quit what we start far too early and far too often. Even more than the effort a gritty person puts in on a single day, what matters is that they wake up the next day, and the next, ready to get on that treadmill and keep going.”

I know one thing for certain: don’t settle for less than what you’re capable of, but strive for something bigger. Some of you reading this might identify with this message because it resonates with you on a deeper level. For others, at the end of their tether the message might be nothing more than a trivial pep talk. What I wish to convey irrespective of where you are in your journey is: NEVER settle for less. If you settle for less, you will receive less than you deserve and convince yourself you are justified to receive it.


“Two people on a precipice over Yosemite Valley” by Nathan Shipps on Unsplash
Develop A Powerful Vision Of What You Want
“Your problem is to bridge the gap which exists between where you are now and the goal you intend to reach.” — Earl Nightingale
I recall a passage my father often used growing up in 1990s: “Don’t tell me your problems unless you’ve spent weeks trying to solve them yourself.” That advice has echoed in my mind for decades and became my motivator. Don’t leave it to other people or outside circumstances to motivate you because you will be let down every time. It must come from within you. Gnaw away at your problems until you solve them or find a solution. Problems are not stop signs, they are advising you that more work is required to overcome them. Most times, problems help you gain a skill or develop the resources to succeed later. So embrace your challenges and develop the grit to push past them instead of retreat in resignation. Where are you settling in your life right now? Could you be you playing for bigger stakes than you are? Are you willing to play bigger even if it means repeated failures and setbacks? You should ask yourself these questions to decide whether you’re willing to put yourself on the line or settle for less. And that’s fine if you’re content to receive less, as long as you’re not regretful later.

If you have not achieved the success you deserve and are considering giving up, will you regret it in a few years or decades from now? Only you can answer that, but you should carve out time to discover your motivation for pursuing your goals. It’s a fact, if you don’t know what you want you’ll get what life hands you and it may not be in your best interest, affirms author Larry Weidel: “Winners know that if you don’t figure out what you want, you’ll get whatever life hands you.” The key is to develop a powerful vision of what you want and hold that image in your mind. Nurture it daily and give it life by taking purposeful action towards it.

Vision + desire + dedication + patience + daily action leads to astonishing success. Are you willing to commit to this way of life or jump ship at the first sign of failure? I’m amused when I read questions written by millennials on Quora who ask how they can become rich and famous or the next Elon Musk. Success is a fickle and long game with highs and lows. Similarly, there are no assurances even if you’re an overnight sensation, to sustain it for long, particularly if you don’t have the mental and emotional means to endure it. This means you must rely on the one true constant in your favour: your personal development. The more you grow, the more you gain in terms of financial resources, status, success — simple. If you leave it to outside conditions to dictate your circumstances, you are rolling the dice on your future.

So become intentional on what you want out of life. Commit to it. Nurture your dreams. Focus on your development and if you want to give up, know what’s involved before you take the plunge. Because I assure you, someone out there right now is working harder than you, reading more books, sleeping less and sacrificing all they have to realise their dreams and it may contest with yours. Don’t leave your dreams to chance.
'''

In [ ]:
result = run_summarization(text_str)
print(result)

 Have you experienced this before? Who is right and who is wrong? Neither. It was at that point their biggest breakthrough came. Perhaps all those years of perseverance finally paid off. It must come from within you. Where are you settling in your life right now? Could you be you playing for bigger stakes than you are? So become intentional on what you want out of life. Commit to it. Nurture your dreams.


##Compare similarity matrix and TF-IDF methods

In [ ]:
#similarity matrix
generate_clean_summary_blog("business_001.txt", 2)

Indexes of top ranked_sentence order are  [(0.21293950793362676, ['the', 'firm', 'which', 'be', 'now', 'one', 'of', 'the', 'biggest', 'investors', 'in', 'google', 'benefit', 'from', 'sales', 'of', 'high-speed', 'internet', 'connections', 'and', 'higher', 'advert', 'sales', 'timewarner', 'say', 'fourth', 'quarter', 'sales', 'rise', '2%', 'to', '$111bn', 'from', '$109bn', 'its', 'profit', 'be', 'buoy', 'by', 'one-off', 'gain', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'warner', 'bros', 'and', 'less', 'users', 'for', 'aol']), (0.2091791279867516, ['time', 'warner', 'say', 'on', 'friday', 'that', 'it', 'now', 'own', '8%', 'of', 'search-engine', 'google', 'but', 'its', 'own', 'internet', 'business', 'aol', 'have', 'have', 'mix', 'fortunes', 'it', 'lose', '464000', 'subscribers', 'in', 'the', 'fourth', 'quarter', 'profit', 'be', 'lower', 'than', 'in', 'the', 'precede', 'three', 'quarter', 'however', 'the', 'company', 'say', 'aol', 'underlie', 'profit', 'before', 'exceptional', 'items', 

In [ ]:
#tf-idf
with open('business_001.txt', 'r') as file:
    data = file.read()

result = run_summarization(data)
print(result)

 But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.


In [ ]:
#similarity matrix
generate_clean_summary_blog("business_005.txt", 2)

Indexes of top ranked_sentence order are  [(0.2943405799012111, ['share', 'in', 'uk', 'drink', 'and', 'food', 'firm', 'ally', 'domecq', 'have', 'rise', 'on', 'speculation', 'that', 'it', 'could', 'be', 'the', 'target', 'of', 'a', 'takeover', 'by', 'france', 'pernod', 'ricard']), (0.24294286861867673, ['report', 'in', 'the', 'wall', 'street', 'journal', 'and', 'the', 'financial', 'time', 'suggest', 'that', 'the', 'french', 'spirit', 'firm', 'be', 'consider', 'a', 'bid', 'but', 'have', 'yet', 'to', 'contact', 'its', 'target', 'ally', 'domecq', 'share', 'in', 'london', 'rise', '4%', 'by', '1200', 'gmt', 'while', 'pernod', 'share', 'in', 'paris', 'slip', '12%', 'pernod', 'say', 'it', 'be', 'seek', 'acquisitions', 'but', 'refuse', 'to', 'comment', 'on', 'specifics']), (0.238022928422743, ['pernod', 'takeover', 'talk', 'lift', 'domecq']), (0.22469362305736895, ['pernod', 'last', 'major', 'purchase', 'be', 'a', 'third', 'of', 'us', 'giant', 'seagram', 'in', '2000', 'the', 'move', 'which', 'pr

In [ ]:
#tf-idf
with open('business_005.txt', 'r') as file:
    data = file.read()

result = run_summarization(data)
print(result)

 Pernod said it was seeking acquisitions but refused to comment on specifics. The other two-thirds of Seagram was bought by market leader Diageo. The WSJ said that the two were ripe for consolidation, having each dealt with problematic parts of their portfolio.
